In [1]:
%run imports

In [3]:
def detect_phasic(lfp, intervals, th):
    phasic = nts.IntervalSet(0,0)
    for s, e in intervals.iloc:
        inter_ = nts.IntervalSet(s, e)
        t, f, Sxx = bk.signal.wavelet_spectrogram_intervals(lfp,inter_,1, 0.5, 250, 100)
        
        broad_power = nts.Tsd(t,np.nanmean(Sxx,0),'s')
        z_broad_power = bk.compute.nts_zscore(broad_power)

        d = z_broad_power.values > th
        phasic_ = bk.compute.toIntervals(t,d,'s')
        phasic = phasic.union(phasic_)
    phasic = phasic.merge_close_intervals(200,'ms')
    phasic = phasic.drop_short_intervals(500,'ms').reset_index(drop = True)

    return phasic

In [19]:
def write_phasic_tonic(base_folder,local_folder,*args,**kwargs):
    bk.load.current_session_linux(base_folder,local_folder)
    states = bk.load.states(True)

    lfp = {}
    lfp['Hpc'] = bk.load.lfp_in_intervals(bk.load.ripple_channel(),states['REM'])
    phREM = detect_phasic(lfp['Hpc'],states['REM'],2)
    toREM = states['REM'].set_diff(phREM)

    phREM['state'] = 'phREM'
    toREM['state'] = 'toREM'

    phasic = pd.concat((phREM,toREM))

    phasic.to_csv('Intervals/phasic_tonic.csv')

In [20]:
write_phasic_tonic('/mnt/electrophy/Gabrielle/GG-Dataset-Light/','Rat08/Rat08-20130713')

Rat : 8 on day : 6
Working with session Rat08-20130713 @ /mnt/electrophy/Gabrielle/GG-Dataset-Light/Rat08/Rat08-20130713


/home/billel/pCloudDrive/IFM/work/Code/Python/Billel/neuroseries/interval_set.py:66: UserWarning: start is not sorted
  warn(msg, UserWarning)


In [21]:
bk.load.batch(write_phasic_tonic)

100%|██████████| 61/61 [1:15:34<00:00, 74.33s/it]

Batch finished in 4534.265228509903
Some session were not processed correctly
['Rat08-20130718', 'Rat08-20130719', 'Rat08-20130720', 'Rat08-20130722', 'Rat09-20140324', 'Rat09-20140325', 'Rat09-20140326', 'Rat09-20140327', 'Rat09-20140328', 'Rat09-20140329', 'Rat09-20140331', 'Rat09-20140401', 'Rat09-20140402', 'Rat09-20140403', 'Rat09-20140404', 'Rat09-20140405', 'Rat09-20140407', 'Rat09-20140408', 'Rat09-20140409', 'Rat11-20150402', 'Rat11-20150403']
34.42622950819672 %


({'Rat08-20130708': None,
  'Rat08-20130709': None,
  'Rat08-20130710': None,
  'Rat08-20130711': None,
  'Rat08-20130712': None,
  'Rat08-20130713': None,
  'Rat08-20130715': None,
  'Rat08-20130716': None,
  'Rat08-20130717': None,
  'Rat10-20140619': None,
  'Rat10-20140620': None,
  'Rat10-20140622': None,
  'Rat10-20140624': None,
  'Rat10-20140626': None,
  'Rat10-20140627': None,
  'Rat10-20140628': None,
  'Rat10-20140629': None,
  'Rat10-20140701': None,
  'Rat10-20140702': None,
  'Rat10-20140703': None,
  'Rat10-20140704': None,
  'Rat10-20140705': None,
  'Rat10-20140707': None,
  'Rat10-20140708': None,
  'Rat11-20150310': None,
  'Rat11-20150312': None,
  'Rat11-20150313': None,
  'Rat11-20150314': None,
  'Rat11-20150315': None,
  'Rat11-20150316': None,
  'Rat11-20150317': None,
  'Rat11-20150321': None,
  'Rat11-20150323': None,
  'Rat11-20150325': None,
  'Rat11-20150326': None,
  'Rat11-20150327': None,
  'Rat11-20150328': None,
  'Rat11-20150330': None,
  'Rat11-201

In [18]:
phREM['state'] = 'phREM'
toREM['state'] = 'toREM'

pd.concat((phREM,toREM))

,start,end,event,state
0,1927040000,1928923200,phREM,phREM
1,2432904800,2433468000,phREM,phREM
2,2519546400,2520676000,phREM,phREM
3,2543480000,2544552800,phREM,phREM
4,2593708000,2594536800,phREM,phREM
...,...,...,...,...
102,21630000000,21633331199,NaN,toREM
103,21633868799,21674000000,NaN,toREM
104,21851000000,21857498399,NaN,toREM
105,21858067999,21945161599,NaN,toREM


In [5]:
phasic = detect_phasic(lfp['Hpc'],states['REM'],2)
phREM = phasic
toREM = states['REM'].set_diff(phREM)

/home/billel/pCloudDrive/IFM/work/Code/Python/Billel/neuroseries/interval_set.py:66: UserWarning: start is not sorted
  warn(msg, UserWarning)


In [8]:
Sxx = {}
for stru in ['BLA','Hpc']:
    t, f, Sxx[stru] = bk.signal.wavelet_spectrogram_intervals(lfp[stru],states['REM'],2, 0.5, 25, 20)

z_sm_broad_power = {}
for stru in ['BLA','Hpc']:
    broad_power = nts.Tsd(t,np.nanmean(Sxx[stru],0),'s')
    z_broad_power = bk.compute.nts_zscore(broad_power)
    z_sm_broad_power[stru] = bk.compute.nts_smooth(z_broad_power,500,1)

In [9]:
%matplotlib qt

fig, ax = plt.subplots(2, 1, sharex=True)
# ax[0].pcolormesh(t, f, Sxx['Hpc'], shading='gouraud')
ax_power = ax[0].twinx()
# ax_power.plot(z_sm_broad_power['Hpc'].as_units('s'), 'w')
# plt.xlim(4500, 5500)
ax[0].set_ylim(0, 25)
ax[1].plot(lfp['Hpc'].as_units('s'))
bk.plot.intervals(phasic,ax=ax[1])

In [12]:
f,Sxx_ph = scipy.signal.welch(lfp['BLA'].restrict(phREM),fs = 1250,nperseg = 1250,noverlap = 625)
f,Sxx_to = scipy.signal.welch(lfp['BLA'].restrict(toREM),fs = 1250,nperseg = 1250,noverlap = 625)

plt.plot(f,np.log(Sxx_to))
plt.plot(f,np.log(Sxx_ph))

plt.xlim(0,200)
plt.ylim(-3,8)
plt.legend(['toREM','phREM'])
plt.show()

In [ ]:
bk.load.current_session_linux()
neurons, metadata = bk.load.spikes()
t = np.mean(phREM.as_units('s').iloc[:],1)
win, psth_ph = bk.compute.psth(neurons,t.values,0.010,[-3,3])

%matplotlib qt
fig,ax = plt.subplots(1)
y = psth_ph[:,(metadata.Region == 'Hpc') &(metadata.Type == 'Pyr')]
bk.plot.confidence_intervals(win,y.T,'blue',ax)
y = psth_ph[:,(metadata.Region == 'BLA') & (metadata.Type == 'Pyr')]
bk.plot.confidence_intervals(win,y.T,'green',ax)


Rat : 8 on day : 6
Working with session Rat08-20130713 @ /mnt/electrophy/Gabrielle/GG-Dataset-Light/Rat08/Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy
You are using 0.01 ms bins with the function fast off. Consider using "Fast = True" in order to speed up the computations


81it [00:00, 2695.91it/s]


IndexError: boolean index did not match indexed array along dimension 0; dimension is 600 but corresponding boolean dimension is 252

In [32]:
from importlib import reload
reload(bk.signal)

<module 'bk.signal' from '/home/billel/pCloudDrive/IFM/work/Code/Python/Billel/bk/signal.py'>

In [36]:
filt = {}
filt['Hpc'] = bk.signal.passband(lfp['Hpc'],6,12)

inst_fq = bk.signal.instantaneous_frequency(filt['Hpc'],1250)

In [37]:
power,phase = bk.signal.hilbert(filt['Hpc'])

In [54]:
fig,ax = plt.subplots(1)
ax.plot(bk.compute.nts_smooth(inst_fq,1250,125).as_units('s'))
bk.plot.intervals(phREM,'orange',ax = ax)